# Get SVHN data

In [33]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from PIL import Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
import cPickle as pickle

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

In [34]:
###Load SVHN Data files, including mat data, for train and test datasets.

In [35]:
url = 'http://ufldl.stanford.edu/housenumbers/'
last_percent_reported = None

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 1% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or not os.path.exists(filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

train_filename = maybe_download('train.tar.gz',404141560)
test_filename = maybe_download('test.tar.gz',276555967)

Found and verified train.tar.gz
Found and verified test.tar.gz


### Extract files

In [36]:
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall()
    tar.close()
  data_folders = root
  
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

train already present - Skipping extraction of train.tar.gz.
train
test already present - Skipping extraction of test.tar.gz.
test


In [37]:
### Format .mat data

In [38]:
import h5py
import time 

class ImageData:
    def __init__(self, data_folder, imageDataMatFilePath):
        self.data_folder=data_folder
        self.file = os.path.join(data_folder, 'digitStruct.mat')
        self.file = h5py.File(self.file, 'r')
        self.names = self.file['digitStruct/name']
        self.boxes = self.file['digitStruct/bbox']
    
    def get_labels_and_dataset(self):
        image_vertices_count=4
        
        ''' 5 places + 1 for length'''
        digit_count=6
        desired_image_size=64

        #Images to be processed
        image_count = len(self.names)-5000

        #Array to hold labels of digits in an image
        #Add one for index
        image_labels_array=np.zeros(image_count*(digit_count+1)).reshape(image_count,digit_count+1)
        image_vertices_array=np.zeros(image_count*(image_vertices_count+1)).reshape(image_count,image_vertices_count+1)
        dataset = np.ndarray(shape=(image_count, desired_image_size, desired_image_size),dtype=np.float32)
        print("image_labels: ",image_labels_array.shape, "image_dimentions: ",image_vertices_array.shape)
        start_time = time.time()


        ##Iterate over all images and get dims of images's bbox ( and not digit bbox), and digit labels.
        for count in range(image_count):
            left=0.
            top=0.
            bottom=0.
            right=0.
            img_label=self.file[self.boxes[count].item()]["label"]
            img_left=self.file[self.boxes[count].item()]["left"]
            img_top=self.file[self.boxes[count].item()]["top"]
            img_height=self.file[self.boxes[count].item()]["height"]
            img_width=self.file[self.boxes[count].item()]["width"]
            num_digits=len(img_label)
            
            
            image_labels_array[count,:]=float('NaN')
            image_labels_array[count,0]=count
            ##Get bbox and digits for each image
            for digit_counter in np.arange(num_digits):
                if(num_digits == 1):
                    tmp_label=img_label.value[0]
                    image_labels_array[count,digit_count - num_digits + digit_counter +1]=tmp_label
                    tmp_left=img_left.value[0]
                    tmp_top=img_top.value[0]
                    tmp_height=img_height.value[0]
                    tmp_width=img_width.value[0]
                else:
                    tmp_label=self.file[img_label.value[digit_counter].item()].value[0]
                    image_labels_array[count,digit_count - num_digits + digit_counter+1]=tmp_label
                    tmp_left=self.file[img_left.value[digit_counter].item()].value[0]
                    tmp_top=self.file[img_top.value[digit_counter].item()].value[0]
                    tmp_height=self.file[img_height.value[digit_counter].item()].value[0]
                    tmp_width=self.file[img_width.value[digit_counter].item()].value[0]
                tmp_bottom=tmp_top+tmp_height
                tmp_right=tmp_left+tmp_width

                #Find the largest margins by taking min of top and left, and max of right and bottom
                #Also reduce top and left margins, and increase right and bottom margings to avoid cutting digits
                if(left==0 or tmp_left < left):
                    left=tmp_left*0.95;
                if(top==0 or tmp_top < top):
                    top=tmp_top*0.95
                if(bottom ==0 or tmp_bottom > bottom):
                    bottom=tmp_bottom*1.02
                if(right==0 or tmp_right > right):
                    right=tmp_right*1.02
            image_vertices_array[count]=[count,left,top,right,bottom]
            fullname = os.path.join(self.data_folder, str(count+1)+".png")
            im = Image.open(fullname)
            im = im.crop((left, top, right, bottom)).resize([desired_image_size,desired_image_size], Image.ANTIALIAS).convert('L')
            dataset[count]=im
        print("image_labels_array: ",image_labels_array.shape, "\n shape: ",image_labels_array[0:2,:])
        print("dataset: ",dataset.shape)
        print("Time taken to find image bounding box:\n- %4.4f seconds ---" % (time.time() - start_time))
        print("\nsample:\n",dataset[0:1,:])
        return image_labels_array, dataset




In [39]:
train_image_data = ImageData(train_folders, 'digitStruct.mat')
train_image_labels, train_dataset=train_image_data.get_labels_and_dataset()


image_labels:  (28402, 7) image_dimentions:  (28402, 5)
image_labels_array:  (28402, 7) 
 shape:  [[  0.  nan  nan  nan  nan   1.   9.]
 [  1.  nan  nan  nan  nan   2.   3.]]
dataset:  (28402, 64, 64)
Time taken to find image bounding box:
- 339.1290 seconds ---

sample:
 [[[  77.   74.   77. ...,   79.   80.   82.]
  [  75.   74.   76. ...,   79.   80.   81.]
  [  74.   75.   78. ...,   78.   79.   80.]
  ..., 
  [ 100.  104.  107. ...,  101.  102.  102.]
  [ 101.  102.  103. ...,  102.  103.  105.]
  [ 100.  102.  103. ...,  104.  101.  101.]]]


In [40]:
test_image_data = ImageData(test_folders, 'digitStruct.mat')
test_image_labels, test_dataset=test_image_data.get_labels_and_dataset()

image_labels:  (8068, 7) image_dimentions:  (8068, 5)
image_labels_array:  (8068, 7) 
 shape:  [[  0.  nan  nan  nan  nan  nan   5.]
 [  1.  nan  nan  nan   2.   1.  10.]]
dataset:  (8068, 64, 64)
Time taken to find image bounding box:
- 77.0977 seconds ---

sample:
 [[[ 79.  79.  79. ...,  79.  79.  79.]
  [ 79.  79.  79. ...,  79.  79.  79.]
  [ 80.  80.  80. ...,  79.  79.  79.]
  ..., 
  [ 90.  89.  87. ...,  90.  90.  90.]
  [ 88.  87.  85. ...,  91.  91.  91.]
  [ 88.  87.  85. ...,  91.  91.  91.]]]


In [41]:
##Pickle Train and Test Sets
train_pickle_file = 'SVHN_basic_train.pickle'

try:
  f = open(train_pickle_file, 'wb')
  save = {
    'train_image_labels': train_image_labels,
    'train_dataset': train_dataset
    }
  pickle.dump(save, f, 1)
  f.close()
except Exception as e:
  print('Unable to save data to', train_pickle_file, ':', e)
  raise

In [42]:
statinfo = os.stat(train_pickle_file)
print('Compressed train pickle size:', statinfo.st_size)

Compressed train pickle size: 466929141


In [43]:
##Pickle Train and Test Sets
test_pickle_file = 'SVHN_basic_test.pickle'

try:
  f = open(test_pickle_file, 'wb')
  save = {
    'test_image_labels': test_image_labels,
    'test_dataset': test_dataset
    }
  pickle.dump(save, f, -1)
  f.close()
except Exception as e:
  print('Unable to save data to', test_pickle_file, ':', e)
  raise

In [44]:
statinfo = os.stat(test_pickle_file)
print('Compressed test pickle size:', statinfo.st_size)

Compressed test pickle size: 132638167


In [45]:
##Unpickle
train_pickle_file = 'SVHN_basic_train.pickle'

with open(train_pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_image_labels = save['train_image_labels']
  train_dataset = save['train_dataset']
  del save  # hint to help gc free up memory
  print('Training set', train_image_labels.shape, train_dataset.shape)


Training set (28402, 7) (28402, 64, 64)


In [46]:
test_pickle_file = 'SVHN_basic_test.pickle'

with open(test_pickle_file, 'rb') as f:
  save = pickle.load(f)
  test_image_labels = save['test_image_labels']
  test_dataset = save['test_dataset']
  del save  # hint to help gc free up memory
  print('Test set', test_image_labels.shape, test_dataset.shape)

Test set (8068, 7) (8068, 64, 64)
